This notebook is intended for comparison of the electrical consumption of simulated buildings (script dc_ro_model.py), considering:
- Different weather conditions
- Retrofit actions
- Abnormal events

The notebook was designed for comparison of 2 different weathers, 2 buildings, 5 anomalies, and 5 retrofit. If the whole set of these simulation configurations has not been employed, the notebook will not provide the desired output. Make sure all the simulated data have been produces (script dc_ro_model.py) and are available in folder datasets/sim_data/from_generator .

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from pathlib import Path
PROJECT_DIR =Path(os.path.abspath('')).parents[1]

sys.path.append(os.fspath(PROJECT_DIR))
from pipeline.definitions import *

from os import listdir
from os.path import isfile, join

import matplotlib as mpl
import colorsys
import matplotlib.colors as mc
import itertools
from pipeline.rc_model_fx import plot_timeseries, plot_energy_signature, fit_energy_signature, calc_PUE

### Select graphic settings

In [ ]:
graph_setting="notebook" #or "article"

In [ ]:
if graph_setting=="article":
    
    #journal-quality parameter settings
    resolution_factor=2
    desired_font=10

elif graph_setting=="notebook":
    resolution_factor=1
    desired_font=12
    
#conversion factors
cm_to_inch=0.393701
classic_proportion=6.4/4.8
golden_rate=1.618

#Elsevier column width is 8.4 cm, double-column width is 17.7 cm (in inches: 3.31 and 6.97)
small_figsize=(resolution_factor*3.31, resolution_factor*3.31/classic_proportion)
big_figsize=(resolution_factor*6.97, resolution_factor*6.97/classic_proportion)

#changings regarding fonttypex
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = "Arial"

font_size=resolution_factor*desired_font

#define colors palette
colors={}
colors["Original"]="dimgray" 
colors["Anomaly"]="crimson"
colors["Retrofit"]="dodgerblue"
colors["hot"]="darksalmon"
colors["cold"]="mediumturquoise"

#define path for figures
figures_path=FIGURES
#check existance of figure path
if not os.path.exists(figures_path):
    print("The selected directory to store figures does not exist")

### Load data

In [ ]:
path=SIM_DATA_NO_FORMAT
datasets = [f for f in listdir(path) if isfile(join(path, f)) and "simData" in f]
buildings=list(set([f.split("_")[2] for f in datasets]))
weathers=list(set([f.split("_")[3] for f in datasets]))
anomalies=list(set([f.split("_")[6] for f in datasets if "anomaly" in f]))
retrofits=list(set([f.split("_")[6] for f in datasets if "retrofit" in f]))

In [ ]:
print("The following buildings are available:")
print(set(buildings))
print("The following weathers are available:")
print(set(weathers))
print("The following anomalies are available:")
print(anomalies)
print("The following retrofits are available:")
print(retrofits)

In [ ]:
if (len(retrofits)<5) or (len(anomalies)<5) or (len(weathers)<2) or (len(buildings)<2):
    print("WARNING: the whole set of simulations is not available. Make sure that the following numbers of different simulations configurations exist:\nBuildings: 2\nWeathers: 2\nAnomalies: 5\n Retrofit: 5.")

#load all the datasets in a dictionary
dfs={}
for building in buildings:
    for weather in weathers:
        file="simData_building_"+building+"_"+weather+"_weather"
        if file in datasets:
            dfs[building, weather, "original"]=pd.read_csv(path+"/"+file)
        for anomaly in anomalies:
            file="simData_building_"+building+"_"+weather+"_weather_anomaly_"+anomaly
            if file in datasets:
                dfs[building, weather, "anomaly_"+anomaly]=pd.read_csv(path+"/"+file)
        for retrofit in retrofits:
            file="simData_building_"+building+"_"+weather+"_weather_retrofit_"+retrofit
            if file in datasets:
                dfs[building, weather, "retrofit_"+retrofit]=pd.read_csv(path+"/"+file)

### Calculate Energy Signatures

Consider that this step can take a while is many casestudies exist!

In [ ]:
#calculate all energy signatures
es={}
for key in dfs.keys():
    es[key]=fit_energy_signature(dfs[key])

### Compare Weathers

In [ ]:
fig= plt.figure()


for building, r in zip(buildings, range(len(buildings))):
    for weather, c in zip(weathers, range(len(weathers))):
        ax=fig.add_subplot(2,2, 1+r*2+c)
        key=(building, weather, "original")
        plot_energy_signature(es[key], ax=ax)
        if r==0:
            ax.set_xticks([])
            ax.xaxis.set_label_position("top")
            ax.set_ylim([80000, 120000])
            if c==0:
                ax.set_xlabel("Hot weather")
            else:
                ax.set_xlabel("Cold Weather")
        else:
            ax.set_xlabel("Temperature [°C]")
            ax.set_ylim([160000, 250000])
        if c!=0:
            ax.set_yticks([])
            ax.yaxis.set_label_position("right")
            if r==0:
                ax.set_ylabel("Low Power Density")
            else:
                ax.set_ylabel("High Power Density")
        else:
            ax.set_ylabel("Power [kW]")
        ax.set_xlim([-5,35])


fig.set_figheight(5)
fig.set_figwidth(5)
plt.tight_layout()

In [ ]:
building="B"
legend=[]

plt.figure()

for weather in weathers:
    df=dfs[building, weather, "original"]
    plt.plot(df.T_ext, color=colors[weather])
    legend.append(weather)

plt.legend(legend)
plt.xlabel("Time step")
plt.ylabel("Temperature [°C]")

In [ ]:
building="B"
legend=[]
PUE={}
plt.figure()

for weather in weathers:
    df=dfs[building, weather, "original"]
    plt.plot(df.P+df.P, color=colors[weather])
    
    legend.append(weather)
    
plt.legend(legend)
plt.ylabel("Electrical Load [kW]")
plt.title("Comparison of the same building, located in Torino and Palermo")

print(PUE)

In [ ]:
monthly_df

In [ ]:
colors

In [ ]:
monthly_df=pd.DataFrame([])
building="B"

for weather in weathers:
    df=dfs[building, weather, "original"]
    monthly_df[weather]=df.reindex(np.linspace(0+8760/24,8760-8760/24, 12)).interpolate().P_cool
    #legend.append(weather)

monthly_df.plot(y=weathers, kind="bar", rot=0, color=[colors[weathers[0]], colors[weathers[1]]])

plt.legend(legend)
plt.title("Comparison of the same building for different weathers")
plt.xlabel("Month")
plt.ylabel("Monthly consumption for cooling")
plt.xticks(ticks=range(0,12), labels=range(1,13))
plt.show()

### Check COP curve

The COP curve should be the same for a same building located under different weather conditions, yet it should differs for retrofit D (sunstitution of cooling machine).

In [ ]:
fig, axs=plt.subplots(2,1, figsize=big_figsize)

building="A"
retrofit="original"

for weather, ax  in zip(weathers, axs):
    df=dfs[building, weather, retrofit]
    ax.scatter(df.T_ext, df.cop, s=1.5, color="gray")
    ax.set_ylabel("COP")
    ax.legend([weather+ " weather"])

fig.suptitle("Performance of cooling")
axs[1].set_xlabel("External Temperature [°C]")

plt.show()

In [ ]:
fig, axs=plt.subplots(2,1, figsize=big_figsize)

building="A"
weather="hot"

for retrofit, ax  in zip(["original", "retrofit_D"], axs):
    df=dfs[building, weather, retrofit]
    ax.scatter(df.T_ext, df.cop, s=1.5, color="gray")
    ax.set_ylabel("COP")
    ax.legend([retrofit])

fig.suptitle("Performance of cooling")
axs[1].set_xlabel("External Temperature [°C]")

plt.show()

## Compare Anomalies

In [ ]:
anomalies_def={}
anomalies_def["A"]="Malfunzionamento Macchina Frigorifera"
anomalies_def["B"]="Manutenzione Rack"
anomalies_def["C"]="Attivazione tasto comfort"
anomalies_def["D"]="Free cooling OFF"
anomalies_def["E"]="Cooling Sytem Severe Misfunctioning"

In [ ]:
def plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes"):

    legend=["Original", "Anomaly"]
    
    df_original=dfs[building, weather, "original"]
    df_anom=dfs[building, weather, "anomaly_"+anomaly]
    
    fig, axs=plt.subplots(1, figsize=big_figsize)
    
    for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
        axs.plot(df.P, color=colors[ts])
    
    
    axs.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    axs.tick_params(labelsize=font_size)
    
    plt.legend(["Original", "Anomaly "+anomaly], fontsize=font_size)
    if title=="Yes":
        plt.title(anomalies_def[anomaly], fontsize=font_size)
    plt.xlabel("Time step [h]", fontsize=font_size)
    plt.ylabel("Electrical Load [kW]", fontsize=font_size)

    if save=="Yes":
        fig.savefig(figures_path+"/anomaly_"+anomaly+"_twoyears_comparative_ts", bbox_inches='tight', dpi=200)

    return None

In [ ]:
building="A"
weather="hot"
anomaly="A"

plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes")

In [ ]:
building="A"
weather="hot"
anomaly="A"

df_original=dfs[building, weather, "original"]
df_anom=dfs[building, weather, "anomaly_A"]
index_anomaly=df[df_anom.real_anomaly==1].index
ext_index=range(index_anomaly.min()-5, index_anomaly.max()+5)

fig, axs=plt.subplots(3, figsize=big_figsize)
#axs[0].set_title("Abnormal cop and loads for a degradation \nof the cooling system performance of "+str(max(df_anom.cop_anom_degr.unique())*100)+"%")
for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
    axs[0].plot(df.loc[ext_index, "cop"], color=colors[ts])
#axs[0].plot(df_anom.loc[ext_index, "cop"])
#ax[0].legend(legend)
    axs[1].plot(df.loc[ext_index, "P_cool"], color=colors[ts])
#axs[1].plot(df_anom.loc[ext_index, "P_cool"])
#axs[1].legend(legend, fontsize=font_size)
    axs[2].plot(df.loc[ext_index, "P"], color=colors[ts])
#axs[2].plot(df_anom.loc[ext_index, "P"])
#ax[2].legend(legend)

for ax in axs:
    ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    ax.tick_params(labelsize=font_size)
    ylims=ax.get_ylim()
    ax.vlines([index_anomaly.min(), index_anomaly.max() ], -100000, 1000000, color="black", linestyle="--")
    ax.set_ylim(ylims)

axs[0].legend(legend, fontsize=font_size)
axs[0].set_ylabel("COP", fontsize=font_size)
axs[1].set_ylabel("Cooling Load [kW]", fontsize=font_size)
axs[2].set_ylabel("Total Load [kW]", fontsize=font_size)
axs[2].set_xlabel("Time step [h]", fontsize=font_size)

for ax in [axs[0], axs[1]]:
    ax.set_xticks([])

plt.tight_layout()
fig.savefig(figures_path+"/sim_anomaly_"+anomaly, bbox_inches='tight', dpi=450)

In [ ]:
building="A"
weather="hot"
anomaly="B"

plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes")

In [ ]:

df_original=dfs[building, weather, "original"]
df_anom=dfs[building, weather, "anomaly_"+anomaly]

fig, axs=plt.subplots(1, figsize=big_figsize)
index_anomaly=df[df_anom.anomaly==1].index
ext_index=range(index_anomaly.min()-5, index_anomaly.max()+5)

for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
    axs.plot(df.loc[ext_index, "P"], color=colors[ts])

ylims=axs.get_ylim()
axs.vlines([index_anomaly.min(), index_anomaly.max() ], -100000, 1000000, color="black", linestyle="--")
axs.set_ylim(ylims)

axs.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
axs.tick_params(labelsize=font_size)

plt.legend(legend, fontsize=font_size)
plt.xlabel("Time step [h]", fontsize=font_size)
plt.ylabel("Electrical Load [kW]", fontsize=font_size)
fig.savefig(figures_path+"/sim_anomaly_"+anomaly, bbox_inches='tight', dpi=450)

In [ ]:
building="A"
weather="cold"
anomaly="C"

plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes")

In [ ]:
df_original=dfs[building, weather, "original"]
df_anom=dfs[building, weather, "anomaly_"+anomaly]

index_anomaly=df[df_anom.anomaly==1].index
ext_index=range(index_anomaly.min()-5, index_anomaly.max()+5)

fig, axs=plt.subplots(2, figsize=big_figsize)
axs[0].plot(df_anom.loc[ext_index, "T_ext"], color="cornflowerblue")

for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
    axs[0].plot(df.loc[ext_index, "T_in"], color=colors[ts])
    axs[1].plot(df.loc[ext_index, "P"], color=colors[ts])

for ax in axs:
    ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    ax.tick_params(labelsize=font_size)
    ylims=ax.get_ylim()
    ax.vlines([index_anomaly.min(), index_anomaly.max() ], -100000, 1000000, color="black", linestyle="--")
    ax.set_ylim(ylims)
    
axs[0].legend(["T_ext", "T_in, Anom", "T_in"], fontsize=font_size)
axs[0].set_ylabel("Temperature [°C]", fontsize=font_size)
axs[1].set_ylabel("Electrical Load [kW]", fontsize=font_size)
axs[0].set_xticks([])
axs[1].set_xlabel("Time step [h]", fontsize=font_size)

plt.tight_layout()
fig.savefig(figures_path+"/sim_anomaly_"+anomaly, bbox_inches='tight', dpi=450)

In [ ]:
building="A"
weather="hot"
anomaly="D"

plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes")

In [ ]:
df_original=dfs[building, weather, "original"]
df_anom=dfs[building, weather, "anomaly_"+anomaly]

index_anomaly=df[df_anom.anomaly==1].index
ext_index=range(index_anomaly.min()-15, index_anomaly.max()+15)

fig, axs=plt.subplots(1, figsize=big_figsize)

for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
    axs.plot(df.loc[ext_index, "P"], color=colors[ts])


axs.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
axs.tick_params(labelsize=font_size)
ylims=axs.get_ylim()
axs.vlines([index_anomaly.min(), index_anomaly.max() ], -100000, 1000000, color="black", linestyle="--")
axs.set_ylim(ylims)

axs.legend(legend, fontsize=font_size)
plt.xlabel("Time step [h]", fontsize=font_size)
plt.ylabel("Electrical Load [kW]", fontsize=font_size)
fig.savefig(figures_path+"/sim_anomaly_"+anomaly, bbox_inches='tight', dpi=450)

In [ ]:
building="A"
weather="cold"
anomaly="E"

plot_anomaly(dfs, building, weather, anomaly, anomalies_def, save="Yes", title="Yes")

In [ ]:
df_original=dfs[building, weather, "original"]
df_anom=dfs[building, weather, "anomaly_"+anomaly]

index_anomaly=df[df_anom.anomaly==1].index
ext_index=range(index_anomaly.min()-5, index_anomaly.max()+5)

fig, axs=plt.subplots(2, figsize=big_figsize)

for df, ts in zip([df_original, df_anom], ["Original", "Anomaly"]):
    axs[0].plot(df.loc[ext_index, "T_in"], color=colors[ts])
    axs[1].plot(df.loc[ext_index, "P"], color=colors[ts])


axs[0].legend(legend, fontsize=font_size)
plt.xlabel("Time step [h]", fontsize=font_size)

for ax in axs:
    ax.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    ax.tick_params(labelsize=font_size)
    ylims=ax.get_ylim()
    ax.vlines([index_anomaly.min(), index_anomaly.max() ], -100000, 1000000, color="black", linestyle="--")
    ax.set_ylim(ylims)
    
axs[0].set_ylabel("Indoor temperature [°C]", fontsize=font_size)
axs[1].set_ylabel("Electrical Load [kW]", fontsize=font_size)
fig.savefig(figures_path+"/sim_anomaly_"+anomaly, bbox_inches='tight', dpi=450)

## Compare Retrofits

In [ ]:
retrofits_def={}
retrofits_def["A"]="Riduzione isolamento involucro"
retrofits_def["B"]="Riduzione assorbività involucro"
retrofits_def["C"]="Modifica temperatura set point"
retrofits_def["D"]="Sostituzione macchina frigorifera"
retrofits_def["E"]="Decommissioning"

In [ ]:
def plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes"):
    df_original=dfs[building, weather, "original"]
    df_retrofit=dfs[building, weather, "retrofit_"+retrofit]
    
    fig, axs=plt.subplots(1, figsize=big_figsize)
    
    for df, ts in zip([df_original, df_retrofit], ["Original", "Retrofit"]):
        axs.plot(df.P, color=colors[ts])
    
    
    axs.grid(True, which='both',axis='both',alpha=0.5, linewidth=0.45)
    axs.tick_params(labelsize=font_size)
    
    plt.legend(["Original", "Retrofit "+retrofit], fontsize=font_size)
    if title=="Yes":
        plt.title(retrofits_def[retrofit], fontsize=font_size)
    plt.xlabel("Time step [h]", fontsize=font_size)
    plt.ylabel("Electrical Load [kW]", fontsize=font_size)

    if save=="Yes":
        fig.savefig(figures_path+"/retrofit_"+retrofit+"_twoyears_comparative_ts", bbox_inches='tight', dpi=200)

    return None

In [ ]:
savings_scenarios={}
PUE_scenarios_df=pd.DataFrame(index=["Buil. A, Torino", "Buil. A, Palermo", "Buil. B, Torino", "Buil. B, Palermo"], columns=["Original", "A", "B", "C", "D", "E"])
rel_savings_scenarios_df=PUE_scenarios_df.copy()

In [ ]:
building="A"
weather="cold"
retrofit="A"

plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes")

In [ ]:
savings=pd.DataFrame()
i=0

for building in ["A", "B"]:
    for weather in ["cold", "hot"]:
        df_original=dfs[building, weather, "original"]
        df_retrofit=dfs[building, weather, "retrofit_A"]
        PUE_original=calc_PUE(df_original.P, df_original.P_tlc)
        PUE_retrofit=calc_PUE(df_retrofit.P, df_retrofit.P_tlc)
                # print("\nBuilding: "+building+ ", Weather: "+weather)
        # print("\nThe retrofit determines a change of PUE from "+str(PUE_original)+" to "+str(PUE_retrofit))
        savings.loc[building, weather]=str(PUE_original[0].round(3))+" to "+str(PUE_retrofit[0].round(3))
        PUE_scenarios_df.iloc[i, :]["A"]=PUE_retrofit
        PUE_scenarios_df.iloc[i, :]["Original"]=PUE_original
        rel_savings_scenarios_df.iloc[i,:]["Original"]=1
        rel_savings_scenarios_df.iloc[i,:]["A"]=df_retrofit.P.sum()/df_original.P.sum()
        i=i+1

display(savings)
savings_scenarios["Retrofit A"]=savings

In [ ]:
building="A"
weather="cold"
retrofit="B"

plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes")

In [ ]:
savings=pd.DataFrame()
i=0

for building in ["A", "B"]:
    for weather in ["cold", "hot"]:
        df_original=dfs[building, weather, "original"]
        df_retrofit=dfs[building, weather, "retrofit_B"]
        PUE_original=calc_PUE(df_original.P, df_original.P_tlc)
        PUE_retrofit=calc_PUE(df_retrofit.P, df_retrofit.P_tlc)
                # print("\nBuilding: "+building+ ", Weather: "+weather)
        # print("\nThe retrofit determines a change of PUE from "+str(PUE_original)+" to "+str(PUE_retrofit))
        savings.loc[building, weather]=str(PUE_original[0].round(3))+" to "+str(PUE_retrofit[0].round(3))
        PUE_scenarios_df.iloc[i, :]["B"]=PUE_retrofit
        rel_savings_scenarios_df.iloc[i,:]["B"]=df_retrofit.P.sum()/df_original.P.sum()
        i=i+1

display(savings)
savings_scenarios["Retrofit B"]=savings

In [ ]:
building="A"
weather="cold"
retrofit="C"

plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes")

In [ ]:
savings=pd.DataFrame()
i=0

for building in ["A", "B"]:
    for weather in ["cold", "hot"]:
        df_original=dfs[building, weather, "original"]
        df_retrofit=dfs[building, weather, "retrofit_C"]
        PUE_original=calc_PUE(df_original.P, df_original.P_tlc)
        PUE_retrofit=calc_PUE(df_retrofit.P, df_retrofit.P_tlc)
                # print("\nBuilding: "+building+ ", Weather: "+weather)
        # print("\nThe retrofit determines a change of PUE from "+str(PUE_original)+" to "+str(PUE_retrofit))
        savings.loc[building, weather]=str(PUE_original[0].round(3))+" to "+str(PUE_retrofit[0].round(3))
        PUE_scenarios_df.iloc[i, :]["C"]=PUE_retrofit
        rel_savings_scenarios_df.iloc[i,:]["C"]=df_retrofit.P.sum()/df_original.P.sum()
        i=i+1

display(savings)
savings_scenarios["Retrofit C"]=savings

In [ ]:
building="A"
weather="cold"
retrofit="D"

plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes")

In [ ]:
savings=pd.DataFrame()
i=0

for building in ["A", "B"]:
    for weather in ["cold", "hot"]:
        df_original=dfs[building, weather, "original"]
        df_retrofit=dfs[building, weather, "retrofit_D"]
        PUE_original=calc_PUE(df_original.P, df_original.P_tlc)
        PUE_retrofit=calc_PUE(df_retrofit.P, df_retrofit.P_tlc)
                # print("\nBuilding: "+building+ ", Weather: "+weather)
        # print("\nThe retrofit determines a change of PUE from "+str(PUE_original)+" to "+str(PUE_retrofit))
        savings.loc[building, weather]=str(PUE_original[0].round(3))+" to "+str(PUE_retrofit[0].round(3))
        PUE_scenarios_df.iloc[i, :]["D"]=PUE_retrofit
        rel_savings_scenarios_df.iloc[i,:]["D"]=df_retrofit.P.sum()/df_original.P.sum()
        i=i+1

display(savings)
savings_scenarios["Retrofit D"]=savings

In [ ]:
building="A"
weather="cold"
retrofit="E"

plot_retrofit(dfs, building, weather, retrofit, anomalies_def, save="Yes", title="Yes")

In [ ]:
savings=pd.DataFrame()
i=0

for building in ["A", "B"]:
    for weather in ["cold", "hot"]:
        df_original=dfs[building, weather, "original"]
        df_retrofit=dfs[building, weather, "retrofit_E"]
        PUE_original=calc_PUE(df_original.P, df_original.P_tlc)
        PUE_retrofit=calc_PUE(df_retrofit.P, df_retrofit.P_tlc)
                # print("\nBuilding: "+building+ ", Weather: "+weather)
        # print("\nThe retrofit determines a change of PUE from "+str(PUE_original)+" to "+str(PUE_retrofit))
        savings.loc[building, weather]=str(PUE_original[0].round(3))+" to "+str(PUE_retrofit[0].round(3))
        PUE_scenarios_df.iloc[i, :]["E"]=PUE_retrofit
        rel_savings_scenarios_df.iloc[i,:]["E"]=df_retrofit.P.sum()/df_original.P.sum()
        i=i+1

display(savings)
savings_scenarios["Retrofit E"]=savings

### Overview of the benefits of retrofits

In [ ]:
fig, axs=plt.subplots(1,2, figsize=(8,4))
axs[0].plot(rel_savings_scenarios_df.T, "-X")
axs[0].legend(rel_savings_scenarios_df.index)
axs[1].plot(PUE_scenarios_df.T, "-X")
axs[1].legend(PUE_scenarios_df.index)
axs[0].set_title("Percentage savings (total energy)")
axs[1].set_title("PUE")